In [1]:
#импорт библиотек, загрузка модели, весов, разметки клаассов
import torch
import numpy as np
import pandas as pd
device = torch.device("cpu")
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, get_scheduler
model_name = "DeepPavlov/rubert-base-cased-sentence"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=660).to(device)

from sentence_transformers import SentenceTransformer
model_sen = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

from sklearn import preprocessing
Label_encoder = preprocessing.LabelEncoder()

#Label_encoder.classes_ = np.load('./cl_classes2610.npy', allow_pickle=True)
#model.load_state_dict(torch.load("../../../best_model2306/pytorch_model.bin", map_location=device))

#после обучения поменять на:

Label_encoder.classes_ = np.load('../sets/classes4d.npy', allow_pickle=True)
model.load_state_dict(torch.load("../../../DeepPavlov_model/pytorch_model.bin", map_location=device))

#импорт описания классов (выдрал с альта софта)
dscr = pd.read_csv("../data/desc4d.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})
dscr10d = pd.read_csv("../data/desc10d.csv", sep=';', names=['id', 'label', 'valid'], dtype={'id': str, 'label': str, 'valid': str})



C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
#функция предсказания одного класса (аргмакс). По умолчанию с кодом класса дает описание класса
def predict_class(text, desc=True):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        if desc==True:
            result = list()
            id = Label_encoder.inverse_transform([predicted_class_id])[0]
            result.append(id)
            result.append(dscr[dscr['id']==id].iloc[0]['label'])
            return result
        else:
            return Label_encoder.inverse_transform([predicted_class_id])[0]

def distance(embeddings1, embeddings2, distance_metric=1):
    if distance_metric==0:# Euclidian
        diff = np.subtract(embeddings1, embeddings2)
        dist = np.sum(np.square(diff),0)
    elif distance_metric==1:#cosine
        dot = np.sum(np.multiply(embeddings1, embeddings2), axis=0)
        norm = np.linalg.norm(embeddings1, axis=0) * np.linalg.norm(embeddings2, axis=0)
        similarity = dot / norm
        dist = np.arccos(similarity) / np.pi
    else:
        raise 'Undefined %d' % distance_metric
    return dist

In [11]:
#загрузим обучающий датасет
#df = pd.read_csv("../sets/train_with_ids.csv")#, sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})
#df = pd.read_csv("../data/firstset.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})
df = pd.read_csv("../sets/train_with_ids.csv", sep=',', dtype={'id': str, 'description': str, 'g_id': str, 'labels': int})
df['label'] = df['description']
df = df.drop(columns=['labels', 'description', 'g_id'])
#функция для вывода обучающих данных по коду. При pandas=False возвращает numpy массив
def whats_data(id, pandas=True):
    if pandas == True:
        return df[df.id.str.slice(start=0, stop=len(str(id)))==str(id)]
    else:
        return df[df.id.str.slice(start=0, stop=len(str(id)))==str(id)].to_numpy()

In [99]:

def makearr(text):
    pred = predict_class(text)
    arr = np.array([[pred[0], pred[1], 0]])
    t_digits = dscr10d.loc[dscr10d.id.str.slice(start=0, stop=4)==pred[0], ['id', 'label']].to_numpy()
    sents = list()
    sents.append(text)
    for each in t_digits:
        arr = np.append(arr, [[each[0], each[1], 0]], axis=0)
        sents.append(each[1])

    embeddings = model_sen.encode(sents)
    return arr, embeddings

def get_sorted(text):
    arr, embs = makearr(text)
    for i in range(1, len(embs)):
        arr[i][2] = distance(embs[0], embs[i])
    a = pd.DataFrame(arr, columns = ['id','label','distance']).sort_values(by=['distance']).style.set_properties(subset=['label'], **{'width': '800px'})
    return a


In [5]:
get_sorted('шприцы очень маленькие')

,id,label,distance
0,9018,"Приборы и устройства, применяемые в медицине, хирургии, стоматологии или ветеринарии, включая сцинтиграфическую аппаратуру, аппаратура электромедицинская прочая и приборы для исследования зрения",0
7,9018311009,"прочие шприцы, с иглами или без игл, из пластмассы",0.22475879623338166
5,9018321000,иглы металлические трубчатые,0.2397155065323802
8,9018311001,"шприцы, с иглами или без игл, из пластмассы, для инсулина объемом не более 2 мл",0.25143066596953373
4,9018390000,"прочие шприцы, иглы, катетеры, канюли и аналогичные инструменты",0.26833115324257945
3,9018191000,прочая аппаратура электродиагностическая для одновременного контроля двух или более параметров,0.28426613635820075
2,9018902000,эндоскопы,0.295797810352401
1,9018908409,"инструменты и оборудование, прочие, применяемые в медицине, хирургии, стоматологии или ветеринарии, включая сцинтиграфическую аппаратуру, аппаратура электромедицинская прочая и приборы для исследования зрения",0.29582308205233004
6,9018901000,инструменты и оборудование для измерения кровяного давления,0.30457541926401827


In [6]:
#тесты с косинусным сходством ()

model_name = "sberbank-ai/sbert_large_nlu_ru"
tokenizer_t = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=655)
model_t = AutoModel.from_pretrained(model_name)

def embed_bert_cls(text, model=model_t, tokenizer=tokenizer_t):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    #model
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()



def get_distance(id1, id2, metric=1):
    #label1 = df['label'][id1] if type(id1) == int else str(id1)
    #label2 = df['label'][id2] if type(id2) == int else str(id2)
    print(f'distance between {label1} and {label2}: {distance(embed_bert_cls(label1), embed_bert_cls(label2), distance_metric=metric)}')

def predict_10d(text, verbose=True):
    pred4 = predict_class(text)
    
    #dscr10d.loc[dscr10d.id.str.slice(start=0, stop=4)==pred4, ['id', 'label']]
    t_digits = dscr10d.loc[dscr10d.id.str.slice(start=0, stop=4)==pred4[0], ['id', 'label']].to_numpy()
    
    cos = dict()
    for each in t_digits:
        cos[each[0]] = distance(embed_bert_cls(text), embed_bert_cls(each[1]))
    #cos
    pred10 = dscr10d[dscr10d['id'] == min(cos, key = cos.get)].to_numpy()
    if verbose == True:
        print(f'predicted 4 digs: {pred4}\n')
        print(f'10 digits options: {t_digits}\n')
        print(f'predicted 10 digs: {pred10}')
    return pred10

prediction = predict_10d("изделия фармацевтические резиновые")

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yeril\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Asking to truncate to max_length but no maximum lengt

predicted 4 digs: ['4017', 'Резина твердая (например, эбонит) во всех формах, включая отходы и скрап; изделия из твердой резины:']

10 digits options: [['4017000009'
  'резина твердая (например, эбонит) во всех формах, включая отходы и скрап; изделия из твердой резины, прочие']]

predicted 10 digs: [['4017000009'
  'резина твердая (например, эбонит) во всех формах, включая отходы и скрап; изделия из твердой резины, прочие'
  nan]]


In [9]:
prediction = predict_10d("презервативы из латекса")

predicted 4 digs: ['4014', 'Изделия гигиенические или фармацевтические (включая соски) из вулканизованной резины, кроме твердой резины, с фитингами из твердой резины или без них']

10 digits options: [['4014100000' 'контрацептивы']
 ['4014900000'
  'изделия гигиенические или фармацевтические (включая соски) из вулканизованной резины, кроме твердой резины, с фитингами из твердой резины или без них, прочие']]

predicted 10 digs: [['4014100000' 'контрацептивы' nan]]


In [89]:
test_df = pd.read_csv("../sets/test_with_ids.csv", dtype={'id': str, 'description': str, 'g_id': str, 'labels': int}) # names=['id', 'description', 'g_id', 'labels'],
test_df = test_df.sample(frac=1).reset_index(drop=True)
test =  test_df[int(len(test_df)*0.8):]

In [42]:
from sklearn import preprocessing
from sklearn.metrics import classification_report
from tqdm import tqdm

In [ ]:
#print(classification_report(Label_encoder.inverse_transform(true), Label_encoder.inverse_transform(preds)))

In [47]:
test =  test_df[int(len(test_df)*0.08):]
test

,id,description,g_id,labels
37,9503003500,игрушки для детей из полимерных материалов наб...,9503,635
38,7616910000,части систем промышленной вентиляции и кондици...,7616,422
39,9503007000,игрушки детские из полимерных материалов в наб...,9503,635
40,3305900009,средства для волос не содержат этиловый спирт ...,3305,119
41,4011400000,новые пневматические резиновые шины dunlop для...,4011,171
...,...,...,...,...
46591,8483308007,подшипники скольжения для валов прочие чехия ...,8483,518
46592,3923409000,катушка из пластмассы для шланга на колесах не...,3923,161
46593,4818201000,изделия из бумаги бытового и санитарно гигиен ...,4818,209
46594,9001900009,зеркало,9001,585


In [90]:
true = test_df['id'][:int(len(test_df)*0.0008)]
labels = test_df['description'][:int(len(test_df)*0.0008)]

In [91]:
preds = list()
for each in tqdm(labels):
    #print(each)
    preds.append(predict_10d(each, verbose=False)[0][0])

100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [04:25<00:00,  7.19s/it]


In [94]:
#print(classification_report(true, preds))

In [ ]:
whats_data(predict_class('покрышки для воздушных судов', desc=False)).style.set_properties(subset=['label'], **{'width': '800px'})

TypeError: 'Styler' object is not subscriptable

In [58]:
def takearr(text):
    pred = predict_class(text)
    arr = np.array([[pred[0], pred[1], 0]])
    t_digits = dscr10d.loc[dscr10d.id.str.slice(start=0, stop=4)==pred[0], ['id', 'label']].to_numpy()
    sents = list()
    sents.append(text)
    for each in t_digits:
        arr = np.append(arr, [[each[0], each[1], 0]], axis=0)
        sents.append(each[1])
    #print(sents)
    embeddings = model_compare.encode(sents)
    return arr, embeddings

In [33]:
df = pd.read_csv("../data/mergedcleared1307.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})
test = pd.read_csv("../../../test.csv", sep=',')#, names=['id', 'label'], dtype={'id': str, 'label': str})
test['id'] = Label_encoder.inverse_transform(test['labels'])
test['label'] = test['description']
test = test.drop(columns=['description'])
test = test.merge(df, on='label', how='left')
test = test[test['id_x']==test.id_y.str.slice(start=0, stop=4)]
test['y'] = test['id_y']
test = test.drop(columns=['labels', 'id_x', 'id_y'])
#после обучения закоментить и поменять на
#test = pd.read_csv("../sets/test_with_ids.csv", sep=',')#, names=['id', 'label'], dtype={'id': str, 'label': str})
#test['i'] = test['id']
#test = test.drop(columns=['labels', 'g_id', 'id'])

In [38]:
test

,description,i
0,полимеры в первичных формах силиконовая резина...,3910000008
1,прочие жернова камни точильные круги шлифоваль...,6804229000
2,картофель молодой свежий калибр 55 для употреб...,701905000
3,шины пневматические резиновые новые для легков...,4011100009
4,аккумуляторы электрические свинцовые гелевые с...,8507208001
...,...,...
46591,декоративные изделия из гипса в виде статуэток,6809900000
46592,переключатели на напряжение не более 60 в проч...,8536501907
46593,сварочные прутки из кремнемарганцовистой стали...,7228209900
46594,принадлежности к одежде капюшон цвета в ассорт...,6117900000


In [70]:
from imblearn.under_sampling import TomekLinks
tl = TomekLinks()

In [71]:
y = data['id']
X = data['label']
X_res, y_res = tl.fit_resample(X, y)

ValueError: could not convert string to float: 'шины и покрышки пневматические резиновые новые для автобусов или моторных транспортных средств для перевозки грузов с индексом нагрузки не более 121 для легких грузовых автомобилей 195 75r16c 107 105r tl mps125 фирма continental matador truck tires s r o  tm matador'

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

knn.fit(X_train, y_train)

preds = knn.predict(X_test)

accuracy_score(y_test, preds)